In [2]:
import cudaq
from cudaq import spin
import networkx as nx
import numpy as np
from typing import Optional
import pandas as pd


In [5]:
!pip freeze

anyio==3.6.2
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arrow==1.2.3
asttokens==2.2.1
attrs==23.1.0
backcall==0.2.0
beautifulsoup4==4.12.2
bleach==6.0.0
cffi==1.15.1
comm==0.1.3
contourpy==1.0.7
cycler==0.11.0
dbus-python==1.2.18
debugpy==1.6.7
decorator==5.1.1
defusedxml==0.7.1
distro==1.7.0
executing==1.2.0
fastjsonschema==2.16.3
fonttools==4.39.3
fqdn==1.5.1
idna==3.4
ipykernel==6.22.0
ipython==8.12.0
ipython-genutils==0.2.0
isoduration==20.11.0
jedi==0.18.2
Jinja2==3.1.2
jsonpointer==2.3
jsonschema==4.17.3
jupyter-events==0.6.3
jupyter_client==8.2.0
jupyter_core==5.3.0
jupyter_server==2.5.0
jupyter_server_terminals==0.4.4
jupyterlab-pygments==0.2.2
jupyterthemes==0.20.0
kiwisolver==1.4.4
lesscpy==0.15.1
MarkupSafe==2.1.2
matplotlib==3.7.1
matplotlib-inline==0.1.6
mistune==2.0.5
nbclassic==0.5.5
nbclient==0.7.3
nbconvert==7.3.1
nbformat==5.8.0
nest-asyncio==1.5.6
notebook==6.5.4
notebook_shim==0.2.2
numpy==1.24.2
packaging==23.1
pandocfilters==1.5.0
parso==0.8.3
pexpect==4.8.0

In [16]:
number_of_qubits = 5
layer_count = 1
parameter_count = 4 * layer_count * number_of_qubits 

In [4]:
coreset_weights = np.random.random(number_of_qubits)
coreset_vectors = np.random.random((number_of_qubits,2))

In [5]:
def get_Hamil_variables(
    coreset_vectors: np.ndarray,
    coreset_weights: np.ndarray,
    index_vals_temp: Optional[int] = None,
    new_df: Optional[pd.DataFrame] = None,
):
    """
    Generates the variables required for Hamiltonian

    Args:
        coreset_vectors: Coreset vectors
        coreset_weights: Coreset weights
        index_vals_temp: Index in the hierarchy
        new_df: new dataframe create for this problem,

    Returns:
       Graph, weights and qubits
    """
    if new_df is not None and index_vals_temp is not None:
        coreset_weights, coreset_vectors = get_cv_cw(coreset_vectors, coreset_weights, index_vals_temp)
    
    coreset_points, G, H, weight_matrix, weights = gen_coreset_graph(
        coreset_vectors, coreset_weights, metric="dot"
    )
    qubits = len(G.nodes)

    return G, weights, qubits

def create_Hamiltonian_for_K2(G, qubits, weights: np.ndarray = None,add_identity=False):
    """
    Generate Hamiltonian for k=2

    Args:
        G: Problem as a graph
        weights: Edge weights
        nodes: nodes of the graph
        add_identity: Add identiy or not. Defaults to False.

    Returns:
        _type_: _description_
    """
    H = 0
#     if add_identity:
#         H = -1 * sum(weights) 
#     else:
#         H = 0 

    for i, j in G.edges():
        weight = G[i][j]["weight"]#[0]
        H += weight * (spin.z(i) * spin.z(j))
        
    return H
        
def gen_coreset_graph(
    coreset_vectors: np.ndarray,
    coreset_weights: np.ndarray,
    metric: str = "dot",
):
    """
    Generate a complete weighted graph using the provided set of coreset points

    Parameters
    ----------
    coreset_weights : ndarray
        np.Coreset weights in array format

    coreset_vectors : ndarray
        Data points of the coreset

    metric : str
        Choose the desired metric for computing the edge weights.
        Options include: dot, dist

    Returns
    -------
    coreset : List((weight, vector))
        The set of points used to construct the graph
    G : NetworkX Graph
        A complete weighted graph
    H : List((coef, pauli_string))
        The equivalent Hamiltonian for the generated graph
    weight_matrix : np.array
        Edge weights of the graph in matrix
    weights : np.array
        Edge weights of the graph in an array

    """

    coreset = [(w, v) for w, v in zip(coreset_weights, coreset_vectors)]

    if coreset is None:
        # Generate a graph instance with sample coreset data
        coreset = []
        # generate 3 points around x=-1, y=-1
        for _ in range(3):
            # use a uniformly random weight
            # weight = np.random.uniform(0.1,5.0,1)[0]
            weight = 1
            vector = np.array(
                [
                    np.random.normal(loc=-1, scale=0.5, size=1)[0],
                    np.random.normal(loc=-1, scale=0.5, size=1)[0],
                ]
            )
            new_point = (weight, vector)
            coreset.append(new_point)

        # generate 3 points around x=+1, y=1
        for _ in range(2):
            # use a uniformly random weight
            # weight = np.random.uniform(0.1,5.0,1)[0]
            weight = 1
            vector = np.array(
                [
                    np.random.normal(loc=1, scale=0.5, size=1)[0],
                    np.random.normal(loc=1, scale=0.5, size=1)[0],
                ]
            )
            new_point = (weight, vector)
            coreset.append(new_point)

    # Generate a networkx graph with correct edge weights
    n = len(coreset)
    G = nx.complete_graph(n)
    H = []
    weights = []
    weight_matrix = np.zeros(len(G.nodes) ** 2).reshape(len(G.nodes()), -1)
    for edge in G.edges():
        pauli_str = ["I"] * n
        # coreset points are labelled by their vertex index
        v_i = edge[0]
        v_j = edge[1]
        pauli_str[v_i] = "Z"
        pauli_str[v_j] = "Z"
        w_i = coreset[v_i][0]
        w_j = coreset[v_j][0]
        if metric == "dot":
            mval = np.dot(coreset[v_i][1], coreset[v_j][1])
        elif metric == "dist":
            mval = np.linalg.norm(coreset[v_i][1] - coreset[v_j][1])
        else:
            raise Exception("Unknown metric: {}".format(metric))

        weight_val = w_i * w_j
        weight_matrix[v_i, v_j] = weight_val
        weight_matrix[v_j, v_i] = weight_val
        G[v_i][v_j]["weight"] = w_i * w_j * mval
        weights.append(w_i * w_j * mval)
        H.append((w_i * w_j * mval, pauli_str))

    return coreset, G, H, weight_matrix, weights

In [6]:
G, weights, qubits = get_Hamil_variables(coreset_vectors, coreset_weights)
H = create_Hamiltonian_for_K2(G=G,qubits=qubits )

In [7]:
def kernel_two_local() -> cudaq.Kernel:
    """QAOA ansatz for maxcut"""
    kernel, thetas = cudaq.make_kernel(list)
    qreg = kernel.qalloc(number_of_qubits)

    # Create superposition
    kernel.h(qreg)

    # Loop over the layers
    theta_position = 0
    
    for i in range(layer_count):
        # Loop over the qubits
        # Problem unitary
        for j in range(number_of_qubits):
            kernel.rz(thetas[theta_position], qreg[j % number_of_qubits])
            kernel.rx(thetas[theta_position + 1], qreg[j % number_of_qubits])
            kernel.cx(qreg[j], qreg[(j + 1) % number_of_qubits])
            kernel.rz(thetas[theta_position + 2], qreg[j % number_of_qubits])
            kernel.rx(thetas[theta_position + 3], qreg[j % number_of_qubits])
            theta_position += 4



    return kernel

In [8]:
optimizer = cudaq.optimizers.COBYLA()
optimizer.initial_parameters = np.random.uniform(-np.pi / 8.0, np.pi / 8.0,
                                                 parameter_count)
print("Initial parameters = ", optimizer.initial_parameters)

Initial parameters =  [-0.26168081699392554, 0.0702049280659735, -0.20816008092593669, 0.3762134783489175, -0.1788239302771347, -0.3314808964683456, -0.13670929826452588, 0.06461758142632512, -0.10130555794051987, 0.307015521700289, -0.1167336282687112, -0.32456216226518564, 0.030513638622854, -0.3713744810285651, 0.38688848381494545, 0.367631645920499, 0.3004983087124502, 0.09263209879673434, 0.22138671039320212, -0.11192750825468734]


In [9]:
optimal_expectation, optimal_parameters = cudaq.vqe(
    kernel=kernel_two_local(),
    spin_operator=H,
    optimizer=optimizer,
    parameter_count=parameter_count)

# Print the optimized value and its parameters
print("Optimal value = ", optimal_expectation)
print("Optimal parameters = ", optimal_parameters)

# Sample the circuit using the optimized parameters
counts = cudaq.sample(kernel_two_local(), optimal_parameters)

<H> = -0.000318
<H> = 0.002255
<H> = -0.001111
<H> = 0.001126
<H> = -0.000194
<H> = -0.004534
<H> = 0.009263
<H> = -0.004972
<H> = 0.002138
<H> = -0.004709
<H> = -0.004434
<H> = -0.002983
<H> = -0.003666
<H> = -0.000297
<H> = -0.005841
<H> = -0.011205
<H> = -0.024952
<H> = -0.023839
<H> = -0.008428
<H> = -0.027206
<H> = 0.023965
<H> = -0.054555
<H> = -0.021985
<H> = -0.051796
<H> = -0.038385
<H> = -0.049930
<H> = -0.049709
<H> = -0.054699
<H> = 0.135142
<H> = -0.054790
<H> = -0.063048
<H> = -0.060858
<H> = -0.062821
<H> = -0.062796
<H> = -0.062810
<H> = -0.062175
<H> = -0.062010
<H> = -0.067497
<H> = -0.059704
<H> = -0.077393
<H> = -0.011662
<H> = -0.085809
<H> = -0.048361
<H> = -0.085732
<H> = -0.083616
<H> = -0.078278
<H> = 0.006825
<H> = -0.091364
<H> = -0.058216
<H> = -0.092482
<H> = -0.069219
<H> = -0.092444
<H> = -0.076163
<H> = -0.092161
<H> = -0.081783
<H> = -0.094639
<H> = -0.096896
<H> = -0.083378
<H> = -0.097205
<H> = -0.100484
<H> = -0.104761
<H> = -0.086467
<H> = -0.110279

In [11]:
print(counts)

{ 11001:517 01001:483 }

